# Classification of the Feature Neurons from the SNN
This notebook performs the classification of the feature neurons from the SNN according to certain criteria. In the future, it would be interesting to implement a 3rd layer in the SNN to classify the feature neurons automatically. For now, this manual classification will classify each neuron as one of the following:
- **Silent Neuron**: Neuron that does not fire at all.
- **Noisy Neuron**: Neuron that fires randomly or without a relevant pattern.
- **Ripple Neuron**: Neuron that fires in the presence of a ripple.
- **Fast Ripple Neuron**: Neuron that fires in the presence of a fast ripple.

## Check WD (change if necessary) and file loading

In [74]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/snn" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/snn"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/snn


## Load the Voltage and Current Dynamics during the SNN run

#### Load the Baseline Thresholds from the output file from the baseline process

In [77]:
import numpy as np
from utils.io import preview_np_array
from utils.input import MarkerType, band_to_file_name

DATASET_FILENAME = "seeg_filtered_subset_90-119_segment500_200"

# Load the Baseline Thresholds
BASELINE_FILE = f"../signal_to_spike/baseline_results/{DATASET_FILENAME}_thresholds.npy"
baseline_thresholds = np.load(BASELINE_FILE)

# preview_np_array(baseline_thresholds, "baseline_thresholds", edge_items=3)

baseline_ripple_thresh = round(baseline_thresholds[0], 4)
baseline_fr_thresh = round(baseline_thresholds[1], 4)

# For now, the UP and DN thresholds are the same (symmetric)
ripple_thresh_up = baseline_ripple_thresh
ripple_thresh_down = -baseline_ripple_thresh
fr_thresh_up = baseline_fr_thresh
fr_thresh_down = -baseline_fr_thresh

print("Ripple Thresholds: ", ripple_thresh_up, ripple_thresh_down)
print("FR Thresholds: ", fr_thresh_up, fr_thresh_down)

Ripple Thresholds:  1.8974 -1.8974
FR Thresholds:  1.0976 -1.0976


In [75]:
from utils.input import MarkerType

def band_to_thresholds(band: MarkerType):
    """Returns the ripple and FR thresholds for the given band

    Args:
        band (MarkerType): the band for which the thresholds are required
    """
    if band == MarkerType.RIPPLE:
        return ripple_thresh_up, ripple_thresh_down
    elif band == MarkerType.FAST_RIPPLE:
        return fr_thresh_up, fr_thresh_down
    else:
        raise Exception("Invalid band type")

In [76]:
# Declare if using ripples, fast ripples, or both
chosen_band = MarkerType.FAST_RIPPLE     # RIPPLE, FAST_RIPPLE, or BOTH
band_file_name = band_to_file_name(chosen_band)

# Define the chosen thresholds
thresh_up, thresh_down = band_to_thresholds(chosen_band)
print("Thresholds: ", thresh_up, thresh_down)

INPUT_PATH = f"./results/{DATASET_FILENAME}"
TIME_SUFFIX = "time0-120000-1"
THRESH_SUFFIX = f"thresh{thresh_up}-{thresh_down}"

# Load the voltage and current data from the numpy files
voltage_file_name = f"{INPUT_PATH}/{band_file_name}_v_dynamics_0.07dv_5ch_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"
current_file_name = f"{INPUT_PATH}/{band_file_name}_u_dynamics_0.07dv_5ch_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"

v_dynamics = np.load(voltage_file_name)
u_dynamics = np.load(current_file_name)

preview_np_array(v_dynamics, "v_dynamics", edge_items=3)
preview_np_array(u_dynamics, "u_dynamics", edge_items=3)

Thresholds:  1.0976 -1.0976
v_dynamics Shape: (120000, 256).
Preview: [[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [-0.02414083  0.37920304  0.12409725 ...  0.16708479  0.0915319
   0.3113081 ]
 [-0.02244321  0.40760014  0.11633999 ...  0.04048213  0.0304657
   0.26017969]
 [ 0.27912785  0.70110552  0.40861676 ...  0.26254686  0.31380562
   0.54026926]]
u_dynamics Shape: (120000, 256).
Preview: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [-2.98474423e-01 -1.61321957e-01 -2.65364602e-01 ... -1.74892075e-01
  -1.62150690e-01 -2.03080634e-0

## Load the SNN Configuration and extract the fields
The SNN configuration contains:
- Numpy array with the ground_truth for each timestep
- Initial Time Offset
- Virtual Time Step Interval
- Number of Steps

In [78]:
from utils.snn import SNNSimConfig

# Load the SNN Config data
snn_config_file_name = f"{INPUT_PATH}/{band_file_name}_snn_config_{TIME_SUFFIX}_{THRESH_SUFFIX}.npy"

# Load the SNNConfig data as an element of the class SNNConfig
snn_config: SNNSimConfig = np.load(snn_config_file_name, allow_pickle=True).item()

# Extract the data fields from the SNNConfig object
ground_truth: np.ndarray = snn_config.ground_truth
init_offset = snn_config.init_offset
virtual_time_step_interval = snn_config.virtual_time_step_interval
num_steps = snn_config.num_steps

preview_np_array(ground_truth, "ground_truth", edge_items=3)
np.count_nonzero(ground_truth)

print("init_offset: ", init_offset)
print("virtual_time_step_interval: ", virtual_time_step_interval)
print("num_steps: ", num_steps)

ground_truth Shape: (199,).
Preview: [('Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Fast-Ripple',   3770.02, 0.) ... ('Fast-Ripple', 116096.  , 0.)
 ('Ripple+Fast-Ripple', 116769.  , 0.) ('Fast-Ripple', 119000.  , 0.)]
init_offset:  0
virtual_time_step_interval:  1
num_steps:  120000


## Find the timesteps where the network spiked
Let's find the timesteps where the network spiked and create a `dictionary` mapping each feature neuron to the timesteps where it spiked.

In [79]:
from utils.data_analysis import find_spike_times

# Create a map storing the spike times for each feature neuron
neuron_spike_times = {}

# Call the find_spike_times util function that detects the spikes in a voltage array
spike_times_lif1 = find_spike_times(v_dynamics, u_dynamics)

total_spikes_count = 0
for (spike_time, neuron_idx) in spike_times_lif1:
    # Calculate the spike time in ms
    real_spike_time = init_offset + spike_time * virtual_time_step_interval

    # If the neuron index is not in the map (first time the feature neuron spikes), add it
    if neuron_idx not in neuron_spike_times:
        neuron_spike_times[neuron_idx] = [real_spike_time]
    # Otherwise, append the spike time to the list of spike times for that neuron
    else:
        neuron_spike_times[neuron_idx].append(real_spike_time)
    total_spikes_count += 1

    # print(f"Spike time: {real_spike_time} (iter. {spike_time}) at neuron: {neuron_idx}")

# Print the spike times for each feature neuron
print("neuron_spike_times: ", neuron_spike_times)


neuron_spike_times:  {97: [20, 724, 812, 1012, 1030, 1478, 1951, 2250, 2325, 3797, 4161, 4324, 4526, 4798, 5168, 5214, 5643, 5676, 5859, 6141, 6206, 6417, 6519, 6655, 7247, 7249, 8120, 8453, 8499, 9119, 9121, 10232, 10386, 10527, 10841, 11324, 11652, 13471, 14055, 14601, 14743, 14925, 15967, 16352, 17061, 17633, 17723, 17766, 18023, 18634, 18732, 18761, 18799, 20124, 20176, 20391, 20445, 20774, 21715, 21798, 21809, 21938, 22676, 22891, 23040, 23047, 23662, 23950, 24225, 24227, 24694, 25041, 25640, 26134, 26671, 26676, 27070, 27415, 27635, 28104, 28207, 28549, 29284, 29782, 30468, 30735, 30949, 31001, 31035, 32124, 32652, 33095, 33732, 33978, 34066, 34594, 34968, 35665, 36483, 38203, 38310, 38564, 39508, 39818, 40536, 40846, 41252, 41467, 42302, 42544, 42902, 42945, 42961, 44772, 45170, 45174, 45561, 46528, 46561, 47123, 47125, 47803, 48190, 48252, 48407, 48464, 48572, 49089, 49284, 49902, 50290, 50746, 51501, 52481, 52590, 53091, 53186, 53218, 53433, 53670, 54071, 54118, 54153, 54466, 

## Define the Time Window after the event insertion to consider as part of the event

In [80]:
from utils.input import RIPPLE_CONFIDENCE_WINDOW, FR_CONFIDENCE_WINDOW, BOTH_CONFIDENCE_WINDOW

# Define the time window after an HFO insertion to consider as part of the event
confidence_window = RIPPLE_CONFIDENCE_WINDOW
if chosen_band == MarkerType.FAST_RIPPLE:
    confidence_window = FR_CONFIDENCE_WINDOW
elif chosen_band == MarkerType.BOTH:
    confidence_window = BOTH_CONFIDENCE_WINDOW

## Merge the spikes that occur at a distance less than the confidence window
In order to calculate the efficiency of the feature neurons, we need to merge the spikes that occur at a distance less than the time window. Otherwise, we would be counting the same event multiple times, which would lead to an overestimation of the efficiency.

This is **probably** safe, since each annotated event occurs at a distance greater than the considered time window, usually at least 1 order of magnitude greater.

In [81]:
# Merge the spikes that occur at a distance less than the confidence window
merged_spikes_counter = 0
for neuron_key in neuron_spike_times.keys():
    spike_times = neuron_spike_times[neuron_key]

    # Array with the merged spike times
    merged_spike_times = [spike_times[0]]

    curr_idx = 1
    comparison_idx = 0
    while curr_idx < len(spike_times):
        spike_time = spike_times[curr_idx]
        prev_spike_time = spike_times[comparison_idx]

        if spike_time - prev_spike_time < confidence_window:
            # This spike is part of the same event
            curr_idx += 1
            merged_spikes_counter += 1
            continue
        else:
            # This spike is part of a new event
            merged_spike_times.append(spike_time)
            comparison_idx = curr_idx   # Update the comparison index to the new spike time
            curr_idx += 1

    # Update the neuron_spike_times map with the merged spike times
    neuron_spike_times[neuron_key] = merged_spike_times

# Print the spike times for each feature neuron
print("Merged neuron_spike_times: ", neuron_spike_times)

print(f"Merged a total of {merged_spikes_counter} spikes out of {total_spikes_count} spikes")

Merged neuron_spike_times:  {97: [20, 724, 812, 1012, 1478, 1951, 2250, 2325, 3797, 4161, 4324, 4526, 4798, 5168, 5643, 5859, 6141, 6206, 6417, 6519, 6655, 7247, 8120, 8453, 9119, 10232, 10386, 10527, 10841, 11324, 11652, 13471, 14055, 14601, 14743, 14925, 15967, 16352, 17061, 17633, 17723, 18023, 18634, 18732, 18799, 20124, 20391, 20774, 21715, 21798, 21938, 22676, 22891, 23040, 23662, 23950, 24225, 24694, 25041, 25640, 26134, 26671, 27070, 27415, 27635, 28104, 28207, 28549, 29284, 29782, 30468, 30735, 30949, 31035, 32124, 32652, 33095, 33732, 33978, 34066, 34594, 34968, 35665, 36483, 38203, 38310, 38564, 39508, 39818, 40536, 40846, 41252, 41467, 42302, 42544, 42902, 44772, 45170, 45561, 46528, 47123, 47803, 48190, 48252, 48407, 48572, 49089, 49284, 49902, 50290, 50746, 51501, 52481, 52590, 53091, 53186, 53433, 53670, 54071, 54153, 54466, 55034, 55309, 55374, 55515, 55775, 56114, 56780, 57059, 57469, 57559, 59018, 59286, 59438, 59529, 59722, 61732, 62094, 64215, 64655, 64788, 65458, 6

## Iterate over the SNN running time and classify the feature neurons

Create a numpy array with the classification of each feature neuron.

In [82]:
from utils.snn import NeuronClass

# Create numpy array containing the class of each feature neuron
feature_neuron_class = np.full(shape=(v_dynamics.shape[1]), fill_value=NeuronClass.SILENT)
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [0 0 0 ... 0 0 0]


### Keep track of the relevant events each feature neuron detected

In [83]:
# Define a map to keep track of the number of relevant events each feature neuron detected
relevant_neuron_spike_times = {}

# Create an empty list for each feature neuron with spikes
for neuron_idx in neuron_spike_times.keys():
    relevant_neuron_spike_times[neuron_idx] = []

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {97: [], 59: [], 47: [], 19: [], 223: [], 17: [], 227: [], 5: [], 44: [], 20: [], 226: [], 30: [], 25: [], 46: [], 22: [], 49: [], 245: [], 146: [], 87: [], 135: [], 220: [], 109: [], 1: [], 205: [], 214: [], 122: [], 3: [], 0: [], 21: [], 6: [], 2: [], 73: [], 104: [], 92: [], 161: [], 112: [], 54: [], 103: [], 66: [], 99: [], 117: [], 11: [], 4: [], 14: [], 35: [], 16: [], 23: [], 27: [], 190: [], 57: [], 250: [], 8: [], 229: [], 120: [], 83: [], 151: [], 64: [], 32: [], 154: [], 13: [], 26: [], 18: [], 252: [], 173: [], 187: [], 158: [], 148: [], 7: [], 45: [], 15: [], 169: [], 106: [], 60: [], 34: [], 213: [], 84: [], 136: [], 162: [], 179: [], 91: [], 9: [], 100: [], 86: [], 69: [], 29: [], 58: [], 36: [], 222: [], 63: [], 72: [], 37: [], 129: [], 50: [], 141: [], 79: [], 124: [], 89: [], 55: [], 237: [], 241: [], 65: [], 207: [], 82: [], 249: [], 176: [], 101: []}


### Define method that searches for a relevant event in the ground_truth np array

In [84]:
# Declare a sorted np.array of the ground truth events by timestamp
ground_truth_timestamps = np.array([ann_event[1] for ann_event in ground_truth])

preview_np_array(ground_truth_timestamps, "ground_truth_timestamps", edge_items=3)

ground_truth_timestamps Shape: (199,).
Preview: [  1000.     3206.54   3770.02 ... 116096.   116769.   119000.  ]


In [85]:
def has_relevant_event(spike_time, confidence_window) -> float | None:
    """
    Searches for a relevant event in the ground truth array given a timestamp.
    The annotated event must be located before the spike time, since the annotation
    corresponds to the insertion of the event.
    Thus, the Event must be in the window [spike_time - confidence_window, spike_time]

    Returns:
    - The insertion time of the relevant event if found
    - None if no relevant event was found
    """
    # Check if spike_time is greater than the first event in the ground truth. If not, return False
    if ground_truth_timestamps[0] > spike_time:
        return None

    for event_time in ground_truth_timestamps:
        if spike_time - confidence_window <= event_time <= spike_time:
            return event_time
        
        if event_time > spike_time:
            # Since the events are sorted, if the event_time is greater than the spike_time, we can stop looking
            return None
    

In [86]:
# Iterate over the feature neurons that spiked
for neuron_idx in neuron_spike_times.keys():
    curr_spike_times = neuron_spike_times[neuron_idx]
    
    # For each Spike
    for curr_spike_time in curr_spike_times:
        # Since the SNN will spike after the insertion of the event, we need to consider the confidence window before the SNN spike.
        # The Annotated event is located at the insertion time (before)

        # Check if the ground truth contains at least 1 annotated event within the confidence window
        if has_relevant_event(curr_spike_time, confidence_window) != None:
            # Add the event to the relevant neuron spike times
            relevant_neuron_spike_times[neuron_idx].append(curr_spike_time)

print("relevant_neuron_spike_times: ", relevant_neuron_spike_times)

relevant_neuron_spike_times:  {97: [1012, 3797, 4161, 4798, 7247, 8120, 9119, 10527, 11652, 14055, 14601, 17061, 18799, 20774, 21798, 22676, 23040, 23662, 24225, 24694, 26671, 27070, 27635, 28549, 29284, 31035, 32124, 33095, 33732, 34066, 38203, 38564, 41252, 42302, 45170, 45561, 47123, 48252, 50290, 50746, 53670, 55034, 55775, 56114, 57559, 59286, 59722, 64655, 66625, 67255, 68064, 69148, 69800, 70777, 72731, 73178, 73745, 74076, 75173, 75746, 76624, 77736, 78284, 79298, 80756, 81741, 83076, 84742, 86643, 91277, 92288, 92635, 93698, 94049, 95197, 95657, 96280, 96721, 97234, 97628, 98077, 99299, 99649, 100595, 101678, 103289, 104807, 106060, 107724, 109529, 113168, 113628, 114316, 119004], 59: [1020, 3240, 3798, 4801, 6701, 7245, 8123, 9117, 10529, 11655, 13221, 14604, 15572, 16155, 16807, 17059, 18800, 20172, 21801, 22677, 24223, 24687, 25139, 25781, 27073, 28245, 28550, 30274, 30767, 31031, 32813, 33088, 33733, 34626, 37214, 38206, 38567, 39243, 44130, 45173, 45562, 46650, 47121, 482

## Statistics regarding the spiking neurons

In [87]:
# Create an array containing the relevant spike ratio for each feature neuron 
# in the order of their keys
relevant_spike_ratios = []
for neuron_idx in neuron_spike_times.keys():
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

    # Calculate the ratio of relevant spikes
    relevant_spike_ratio = (len(relevant_spike_times) / len(curr_spike_times)) * 100 if len(curr_spike_times) > 0 else 0.0
    relevant_spike_ratios.append(relevant_spike_ratio)

In [88]:
PLOT_RELEVANCY_STATS = True

if PLOT_RELEVANCY_STATS:
    # Iterate over the feature neurons that spiked
    for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
        # Get all the spikes of the current neuron
        curr_spike_times = neuron_spike_times[neuron_idx]

        # Get the relevant spikes of the current neuron
        relevant_spike_times = relevant_neuron_spike_times[neuron_idx]

        print(f"Neuron {neuron_idx}: ")
        print(f"Spikes: {curr_spike_times}")
        print(f"Relevant Spikes: {relevant_spike_times}")
        print(f"Relevant Spikes Ratio: {relevant_spike_ratios[iter_idx]}%")
        print("================================================================\n")

Neuron 97: 
Spikes: [20, 724, 812, 1012, 1478, 1951, 2250, 2325, 3797, 4161, 4324, 4526, 4798, 5168, 5643, 5859, 6141, 6206, 6417, 6519, 6655, 7247, 8120, 8453, 9119, 10232, 10386, 10527, 10841, 11324, 11652, 13471, 14055, 14601, 14743, 14925, 15967, 16352, 17061, 17633, 17723, 18023, 18634, 18732, 18799, 20124, 20391, 20774, 21715, 21798, 21938, 22676, 22891, 23040, 23662, 23950, 24225, 24694, 25041, 25640, 26134, 26671, 27070, 27415, 27635, 28104, 28207, 28549, 29284, 29782, 30468, 30735, 30949, 31035, 32124, 32652, 33095, 33732, 33978, 34066, 34594, 34968, 35665, 36483, 38203, 38310, 38564, 39508, 39818, 40536, 40846, 41252, 41467, 42302, 42544, 42902, 44772, 45170, 45561, 46528, 47123, 47803, 48190, 48252, 48407, 48572, 49089, 49284, 49902, 50290, 50746, 51501, 52481, 52590, 53091, 53186, 53433, 53670, 54071, 54153, 54466, 55034, 55309, 55374, 55515, 55775, 56114, 56780, 57059, 57469, 57559, 59018, 59286, 59438, 59529, 59722, 61732, 62094, 64215, 64655, 64788, 65458, 66625, 67000, 

## Plot the Relevant Spike Ratio of the Feature Neurons that Spike

In [89]:
# Create a bar plot containing the % of relevant spikes for each feature neuron 
from utils.bar_plot import create_bar_fig  # Import the function to create the figure

# Define the x and y values
neuron_label = [f"Neu. {neuron_idx}" for neuron_idx in neuron_spike_times.keys()]

# sorting the bars means sorting the range factors
neurons_descending_rel_ratio = sorted(neuron_label, key=lambda x: relevant_spike_ratios[neuron_label.index(x)], reverse=True)

# Create the LIF1 Voltage
feat_neurons_relevancy_plot = create_bar_fig(
    title="Feature Neurons Relevant Spikes Ratio", 
    x_axis_label='Feature Neuron Index', 
    y_axis_label='Relevant Spike Ratio (%)',
    x=neuron_label,
    y=relevant_spike_ratios,
    x_range=neurons_descending_rel_ratio,
    sizing_mode="stretch_width",
    bar_width=0.5,
    tooltips=[("Neuron Index", "@x"), ("Relevant Spike Ratio", "@top%")]
)

In [90]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    # Show the plot
    bplt.show(feat_neurons_relevancy_plot)

In [91]:
EXPORT_RELEVANCY_PLOT = True
OUTPUT_FOLDER = f"./neuron_classification/{DATASET_FILENAME}"

if EXPORT_RELEVANCY_PLOT:
    # Create the output folder if it does not exist
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)

    file_path = f"{OUTPUT_FOLDER}/{band_file_name}_relevancy_barplot_0.07dv_{TIME_SUFFIX}_{THRESH_SUFFIX}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="Feature Neurons Relevancy (%) Bar Plot")

    # Save the plot
    bplt.save(feat_neurons_relevancy_plot)

## Classify the Feature Neurons according to their Spiking Activity
In this step, we will update the `feature_neuron_class` array with the classification of the `Noisy Neurons`, `Ripple Neurons` and `Fast Ripple Neurons`.

We set a **threshold of 0.9** for the `Relevant Spike Ratio` to classify a neuron as a `Ripple` or `Fast Ripple` Neuron.

In [92]:
from utils.snn import marker_type_to_neuron_class

relevant_ratio_threshold = 90.0

# Iterate over the feature neurons that spiked
for iter_idx, neuron_idx in enumerate(neuron_spike_times.keys()):
    # Get all the spikes of the current neuron
    curr_spike_times = neuron_spike_times[neuron_idx]
    # Get the relevant spikes of the current neuron
    relevant_spike_times = relevant_neuron_spike_times[neuron_idx]
    # Get the relevant spikes ratio of the current neuron
    relevant_spike_ratio = relevant_spike_ratios[iter_idx]

    if relevant_spike_ratio >= relevant_ratio_threshold:
        # Classify the Neuron as a RIPPLE/FAST RIPPLE/BOTH detector based on the band we are analyzing
        feature_neuron_class[neuron_idx] = marker_type_to_neuron_class(chosen_band)
    else:
        # Classify the Neuron as Noisy
        feature_neuron_class[neuron_idx] = NeuronClass.NOISY

### Show the classification of the Feature Neurons

In [93]:
preview_np_array(feature_neuron_class, "feature_neuron_class", edge_items=3)

feature_neuron_class Shape: (256,).
Preview: [1 1 1 ... 0 0 0]


# Calculate the Classification Metrics of the SNN based on the Feature Neurons
As of right now, we have a Spiking Neural Network that updates it's state in real-time. This network is capable of detecting relevant oscillatory events through spikes.

Therefore, we have to define the conditions that identify the detection of a HFO event or not. Since each Feature Neuron is modelled to be tuned to a specific frequency range, not all of them will fire simultaneously in the presence of a HFO event. Hence, we need to define a threshold for the number of Feature Neurons that need to fire in order to classify an event as a HFO. This is an exploratory approach, but as a starting point, let's say that we need 2 neurons firing during the time window (`confidence_window`).

In [94]:
# Initialize the variables that store the values of the Confusion Matrix
true_positive = 0
false_positive = 0
true_negative = 0
false_negative = 0

num_consensus_neurons = 1 # 2   # Number of neurons that must agree to classify an event as relevant

### Build a numpy array containing all the spike times from the `Detector Neurons` ordered by spiking time

In [95]:
# Find the index of the Feature Neurons that are HFO(Ripple/FR/Both) Detectors
detector_indices = np.where(feature_neuron_class == marker_type_to_neuron_class(chosen_band))[0]
preview_np_array(detector_indices, "detector_indices")

detector_indices Shape: (7,).
Preview: [  9  36  65  69  89 176 222]


In [96]:
# Build an ordered list of spike times from all the HFO Detector Neurons
detector_spike_times = []

for neuron_idx in detector_indices:
    # print(f"Adding spikes from neuron {neuron_idx}: {neuron_spike_times[neuron_idx]}")
    detector_spike_times.extend(neuron_spike_times[neuron_idx])

# Sort the list of spike times
detector_spike_times.sort()

detector_spike_times = np.array(detector_spike_times)
preview_np_array(detector_spike_times, "detector_spike_times")

detector_spike_times Shape: (18,).
Preview: [ 24201  29275  32790  32804  38185 ...  88178  96279  96718 113162 116798]


In [97]:
import math

curr_detector_idx = 0   # Index of the current detector neuron spike
curr_ground_truth_idx = 0   # Index of the current ground truth event in the ordered array

# Stores an active annotation. If None, there is no active annotation
# An annotation is active in the interval [annotation_start, annotation_start + confidence_window]
active_annotation = None    # Timestamp of the active annotation   

# Go through the voltage dynamics array and calculate the classification metrics of the network
for run_step, feat_neurons_v in enumerate(v_dynamics):
    is_end_detector_spikes = curr_detector_idx >= len(detector_spike_times)
    is_end_ground_truth = curr_ground_truth_idx >= len(ground_truth)

    # Block that checks if a new annotation gets activated
    if not is_end_ground_truth:
        # Check if the next annotation was inserted at this run_step
        next_annotation_insertion = math.ceil(ground_truth_timestamps[curr_ground_truth_idx])
        if next_annotation_insertion == run_step:
            if active_annotation != None:
                raise Exception("There is already an active annotation. This should not happen.")

            # Start a new active annotation
            active_annotation = next_annotation_insertion

            # Increment the curr_ground_truth_idx since this annotation will be processed
            curr_ground_truth_idx += 1

    # If there are still spikes from the Detector Neurons
    if not is_end_detector_spikes:
        next_spike_iter = detector_spike_times[curr_detector_idx]
        # If the next spike occurs in the current run_step interval, let's check what kind of prediction it is
        if next_spike_iter == run_step:
            # Increment the curr_detector_idx since this spike will be processed
            curr_detector_idx += 1

            # Check if there is an annotated event in the ground_truth within the confidence window [run_step, run_step-confidence_window]
            if active_annotation != None:
                # If there is a relevant event, let's see if the SNN predicted it.
                num_votes = 1   # The first vote is the current spike (SNN needs num_consensus_neurons to classify the event as relevant)

                # Calculate the window of detection for this event
                # The annotated events are floats, so we need to ceil the end_relevant_event_time because 
                # we are comparing with the integer run_step (iteration index of the simulation) (TODO: Include the virtual_step in the calculation)
                end_relevant_event_time = math.ceil(active_annotation + confidence_window)

                # Check if there are more spikes in the confidence window
                while curr_detector_idx < len(detector_spike_times) and detector_spike_times[curr_detector_idx] <= end_relevant_event_time:
                    num_votes += 1
                    curr_detector_idx += 1

                # If the number of votes is greater than the consensus threshold, classify the event as relevant
                if num_votes >= num_consensus_neurons:
                    # The SNN detected the event -> True Positive
                    true_positive += 1
                else:
                    # The SNN didn't detect the event (Not enough votes) -> False Negative
                    # False Negatives may also occur without any spikes in the network
                    false_negative += 1

                # The current annotation was processed, so we can deactivate it
                active_annotation = None
            else:
                # If there is no annotated event in the confidence window, let's check if the SNN predicted one.
                # If the SNN predicted an event, it is a False Positive
                # Calculate the window that would cause an SNN Detection
                end_relevant_event_iter = next_spike_iter + confidence_window

                # Verify if there isn't an annotated event between the current run_step and the end_relevant_event_iter
                # If there is an annotated event in between, we need to modify the end_relevant_event_iter to the annotated event time-1
                middle_relevant_event_time = has_relevant_event(end_relevant_event_iter, confidence_window)
                if middle_relevant_event_time != None:
                    # If there is an annotated event in the confidence window, we need to update the end of the window
                    # to the timestep before the annotated event, since we are analyzing the prediction of a False Positive
                    end_relevant_event_iter = math.ceil(middle_relevant_event_time) - 1 

                # Check if there are more spikes in the confidence window
                while curr_detector_idx < len(detector_spike_times) and detector_spike_times[curr_detector_idx] <= end_relevant_event_iter:
                    num_votes += 1
                    curr_detector_idx += 1

                # If the number of votes is greater than the consensus threshold
                if num_votes >= num_consensus_neurons:
                    # The SNN detected an event that does not exist -> False Positive
                    false_positive += 1
                else:
                    # The SNN didn't detect any event -> True Negative
                    true_negative += 1
            
            # Go to the next iteration
            continue
    
    # If one of the 2 conditions is true:
    #   1. There are no more spikes from the Detector Neurons
    #   2. The current run_step does not contain a spike from the Detector Neurons
    # Then: let's check if the active annotation is still active and report a True Negative or False Negative
    # according to the answer.
    
    # Block that checks if the active annotation is still active
    if active_annotation != None:
        if run_step >= active_annotation + confidence_window + 1:   # The +1 is to consider an extra iteration because the annotation is a float ceiled
            # The active annotation is over
            active_annotation = None

            # The SNN didn't detect the event in the interval [active_annotation, active_annotation + confidence_window]
            # Thus, it is a False Negative
            false_negative += 1
        else:
            # The active annotation is still active, so it is a True Negative (No event detected)
            true_negative += 1
    else:
        # If there is no active annotation, the SNN didn't detect any event rightfully so
        # So, it is a True Negative
        true_negative += 1


    # Edge case: If there is an annotated event at a distance < confidence_window from the end of the simulation, the feature events
    # won't have time to detect it. Thus, we don't consider that event.
    # TODO: Consider this edge case?

## Show the Confusion Matrix

In [98]:
# Print the Confusion Matrix
print("True Positive: ", true_positive)
print("False Positive: ", false_positive)
print("True Negative: ", true_negative)
print("False Negative: ", false_negative)

# Print the Total of predictions
total_predictions = true_positive + false_positive + true_negative + false_negative
print("Total Predictions: ", total_predictions)

True Positive:  17
False Positive:  0
True Negative:  119801
False Negative:  182
Total Predictions:  120000


## Calculate Classification Metrics

In [99]:
# Calculate relevant metrics
accuracy = (true_positive + true_negative) / total_predictions * 100    # Proportion of correct predictions
precision = true_positive / (true_positive + false_positive) * 100      # Proportion of TPs that were identified correctly
recall = true_positive / (true_positive + false_negative) * 100         # Proportion of TPs that were captured by the model
f1_score = (2 * precision * recall) / (precision + recall)              # Harmonic mean of Precision and Recall
specificity = true_negative / (true_negative + false_positive) * 100    # Proportion of TNs that were identified correctly

print(f"Accuracy: {accuracy}%")
print(f"Precision: {precision}%")
print(f"Recall: {recall}%")
print(f"F1 Score: {f1_score}")
print(f"Specificity: {specificity}%")

Accuracy: 99.84833333333333%
Precision: 100.0%
Recall: 8.542713567839195%
F1 Score: 15.740740740740739
Specificity: 100.0%


# Export the results of the Classification to a JSON file
Export the results of the classification to a JSON file. This file will include:
- Frequency Band used (`Ripple`, `Fast Ripple` or `Both`)
- `Confidence Window` used
- Classification of each Feature Neuron (`Silent`, `Noisy`, `Ripple`, `Fast Ripple`, `Both`)
- Number of `Consensus Feature Neurons` to classify an event as a HFO
- Classification Metrics (`True Positives`, `False Positives`, `True Negatives`, `False Negatives`, `Accuracy`, `Precision`, `Recall`, `F1 Score`, `Specificity`)

In [100]:
# Calculate the number of feature neurons of each class
num_silent_neurons = np.count_nonzero(feature_neuron_class == NeuronClass.SILENT)
num_noisy_neurons = np.count_nonzero(feature_neuron_class == NeuronClass.NOISY)
num_detector_neurons = np.count_nonzero(feature_neuron_class == marker_type_to_neuron_class(chosen_band))

print(f"Number of Silent Neurons: {num_silent_neurons}")
print(f"Number of Noisy Neurons: {num_noisy_neurons}")
print(f"Number of Detector Neurons: {num_detector_neurons}")
print(f"Total: {num_silent_neurons + num_noisy_neurons + num_detector_neurons}")

Number of Silent Neurons: 150
Number of Noisy Neurons: 99
Number of Detector Neurons: 7
Total: 256


In [101]:
import json

# Export the results to a JSON file

# Create a dictionary with the results
json_results = {
    "freq_band": band_file_name,
    "confidence_window": confidence_window,
    "feature_neuron_class": {
        "array": feature_neuron_class.tolist(),
        "counts": {
            "silent_neurons": num_silent_neurons,
            "noisy_neurons": num_noisy_neurons,
            "detector_neurons": num_detector_neurons
        },
    },
    "num_consensus_neurons": num_consensus_neurons,
    "metrics": {
        "true_positive": true_positive,
        "false_positive": false_positive,
        "true_negative": true_negative,
        "false_negative": false_negative,
        "total_predictions": total_predictions,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1_score,
        "specificity": specificity
    }
}

EXPORT_JSON_FILE = True
if EXPORT_JSON_FILE:
    json_file_name = f"{OUTPUT_FOLDER}/{band_file_name}_results_0.07dv_{TIME_SUFFIX}_{THRESH_SUFFIX}_classif{num_consensus_neurons}.json"
    with open(json_file_name, 'w') as f:
        json.dump(json_results, f)